Title

In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
library(dbplyr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

Introduction:
provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your report
clearly state the question you tried to answer with your project
identify and describe the dataset that was used to answer the question


**from proposal**
>* The data used originated from the Facebook metrics of a popular cosmetics brand. Based on this information, we want to know how successful each post type is based on its audience reach. We will base the sucess of each post by the total number of interactions (likes, comments, shares).
>* We have decided to use the total number of interactions on posts rather than the total amount of times a post has been consumed. This is to determine how much of each post type have the highest engagement/ active users. This is because the number of people viewing a specific post can be misleading because people could be accidently clicking on the post which still counts as 1 view. We want to see the actively viewing/participating users who actually intended on viewing the post. 
* Clearly state the question you will try to answer with your project
>* Which kind of post (link, photo, status or video) is most popular during each day?
In answering the above question we will also think about: which post type is most popular in general? Which day do people interact on facebook the most on?
* Identify and describe the dataset that will be used to answer the question
>* The data set used to answer the question is dataset_Facebook.csv. From the data, the x-axis is the "Total Interaction Count" while the y-axis is the "Weekday Posted." The fill of the bars is the type of post being interacted with. We will use this data to answer the question by finding what kind of post and which day to post is the most popular has the highest value.

Methods & Results:
describe in written English the methods you used to perform your analysis from beginning to end that narrates the code the does the analysis.
your report should include code which:

>loads data from the original source on the web 

>wrangles and cleans the data from it's original (downloaded) format to the format necessary for the planned analysis

>performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis 

>creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis

>performs the data analysis

>creates a visualization of the analysis 
        note: all tables and figure should have a figure/table number and a legend

**from proposal**
>* We will conduct our data analysis by comparing the "Total Interaction Count" of each day which is divided and color-coded by the "Post Type."
>* We will use the Type, Total Interactions and Post Weekday columns as these will be useful in predicting which day and what type of post is most popular. 
* Describe at least one way that you will visualize the results
>* Color-coded bar graph with the x-axis being the "Total Interaction Count" (likes, shares, comments), and the y-axis as the "Weekday Posted." The "Post Type" being color-coded. 
>* Stacked bar graph with each color a different type of post(link, status, photo, video).

Discussion:
    summarize what you found
    discuss whether this is what you expected to find?
    discuss what impact could such findings have?
    discuss what future questions could this lead to?

References
At least 2 citations of literature relevant to the project (format is your choice, just be consistent across the references).
Make sure to cite the source of your data as well.

Sources: (Moro et al., 2016) Moro, S., Rita, P., & Vala, B. (2016). Predicting social media performance metrics and evaluation of the impact on brand building: A data mining approach. Journal of Business Research, 69(9), 3341-3351.
